# Formatting Tables in Pandas


In [20]:
# Import pandas
import pandas as pd
# Import numpy
import numpy as np



## The data
Let's create some simulated data for two widgets, A and B. We'll create a dataframe for each widget, then concatenate them together. We'll also sort the dataframe by month and reset the index.

In [21]:
# simulated data for widget A
df_a = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 1_000_000,
            high = 2_500_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 300_000,
            high = 500_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 750_000,
            high = 1_250_000,
            size = 132
        )
    }
)

df_a['Product'] = 'A'

# simulated data for widget B
df_b = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 100_000,
            high = 800_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 10_000,
            high = 95_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 450_000,
            high = 750_000,
            size = 132
        )
    }
)

df_b['Product'] = 'B'

# put it together & sort
df = pd.concat([df_a,df_b],axis = 0)
df.sort_values(by = 'Month',inplace = True)
df.reset_index(drop = True,inplace = True)

Let’s calculate a few “interesting” statistics — average sale amounts and product conversion:

In [22]:
# average sale
df['Average sale'] = df['Amounts'] / df['Numbers']

# conversion
df['Product conversion'] = df['Numbers'] / df['Quotes']

In [23]:
df.head()

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01-01,1181967,384050,983950,A,2.562036,0.324924
1,2012-01-01,373177,41339,706169,B,17.082392,0.110776
2,2012-02-01,2323529,475012,1225163,A,2.579225,0.204436
3,2012-02-01,514446,40401,520283,B,12.877973,0.078533
4,2012-03-01,1071743,385437,830827,A,2.155546,0.359636


## Date Formatting
There’s arguably nothing --wrong-- with the formatting, but it could be better. For instance, since all the monthly data is reflected as at the first of each month, there’s probably little sense in keeping the day element of each Month entry as it tells the reader very little.

In [32]:
# format the date as YYYY-MM-DD
df.style.format({'Month':'{:%Y-%m}'})
# display(df.head(3))

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01,1181967,384050,983950,A,2.562036,0.324924
1,2012-01,373177,41339,706169,B,17.082392,0.110776
2,2012-02,2323529,475012,1225163,A,2.579225,0.204436
3,2012-02,514446,40401,520283,B,12.877973,0.078533
4,2012-03,1071743,385437,830827,A,2.155546,0.359636
5,2012-03,259326,45706,696434,B,15.237256,0.176249
6,2012-04,2159293,307758,845420,A,2.747029,0.142527
7,2012-04,745427,20448,525969,B,25.722271,0.027431
8,2012-05,1446270,466436,1036636,A,2.222461,0.322510
9,2012-05,439933,34216,691900,B,20.221534,0.077775


Now, we can improve readability even further by using the name of each month rather than the month number, and we can do this *without having to alter the underlying data*.

In [31]:
# format the date as YYYY-MM-DD
df.style.format({'Month':'{:%B %Y}'})
display(df.iloc[:3])

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01-01,1181967,384050,983950,A,2.562036,0.324924
1,2012-01-01,373177,41339,706169,B,17.082392,0.110776
2,2012-02-01,2323529,475012,1225163,A,2.579225,0.204436
